In [37]:
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor
import threading  
from pymongo import MongoClient
# Kết nối với MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['shopeefood']
collection = db['shopeefood']
# Hàm để lấy thông tin chi tiết của delivery_id
def get_nameres(delivery_id):
    url = f"https://gappapi.deliverynow.vn/api/delivery/get_detail?id_type=2&request_id={delivery_id}"

    headers = {
        '3e9cad5f': ';EAE9"^f:K,:Lm3k^"B&0RP$A',
        'a883c2d7': '"S:3r\'.iV]N@aQ-=osIF$j*Sh\'E6=D@eZ;**RS6[RS9^nq8b8(fhM+>*cok:2&RsT3Ch,oL4Sfp8pjKiYq+iln3iaL>0KhrI&11ifCbF]ol1WE/h</\'HQ2I:5TCamq;5TB07id9c]?c/N6OqL9+Ho8\\9uqd.<2G`cZ-rd`?Tnen8Bqr%U]2I)kBnA9n"X_+R>4^%U]2I)kBnA9n"X_+R>4^',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.7',
        'c378317e': 'e1HL_TgdOfrT$Z=5ECOM3:LYk',
        'origin': 'https://shopeefood.vn',
        'priority': 'u=1, i',
        'referer': 'https://shopeefood.vn/',
        'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'sec-gpc': '1',
        'x-foody-access-token': '3db5fa1d24c7ecde485032112fba25ac9100f0b6cc0a8708405caf013476a7628afebcd87238316dcdd6e894a327720b0b6428653480d666a1d47d3d89adbb06',
        'x-foody-api-version': '1',
        'x-foody-app-type': '1004',
        'x-foody-client-id': '',
        'x-foody-client-language': 'vi',
        'x-foody-client-type': '1',
        'x-foody-client-version': '3.0.0',
        'x-sap-ri': 'b0aac8668e7ea8f26dbfa43f7ca764bff832591dd6f01ad2'
    }

    response = requests.get(url, headers=headers)
    data_json = response.json()
    delivery_detail = data_json['reply']['delivery_detail']
    address = delivery_detail['address']
    name = delivery_detail['name']
    return address, name


In [38]:
# Hàm để lấy foody_service_id
def get_foody_service_id(city_id=None):
    url = "https://gappapi.deliverynow.vn/api/meta/get_minimal_metadata"

    headers = {
        'Authority': 'gappapi.deliverynow.vn',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'vi,en-US;q=0.9,en;q=0.8',
        'Cache-Control': 'no-cache',
        'Origin': 'https://shopeefood.vn',
        'Pragma': 'no-cache',
        'Referer': 'https://shopeefood.vn/',
        'Sec-Ch-Ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'X-Foody-Access-Token': '',
        'X-Foody-Api-Version': '1',
        'X-Foody-App-Type': '1004',
        'X-Foody-Client-Id': '',
        'X-Foody-Client-Language': 'vi',
        'X-Foody-Client-Type': '1',
        'X-Foody-Client-Version': '3.0.0'
    }

    response = requests.get(url, headers=headers)
    data_json = json.loads(response.text)

    # Khởi tạo dictionary chứa city_id và danh sách foody_service_id
    city_to_foody_dict = {}

    # Lấy danh sách các thành phố
    cities = data_json['reply']['country']['cities']

    for city in cities:
        cid = city['id']  # Lấy city_id
        foody_service_ids = [service['foody_service_id'] for service in city['search_place_holder_time_ranges']]  # Lấy danh sách foody_service_id
        city_to_foody_dict[cid] = foody_service_ids  # Gán vào dict

    # Nếu city_id được cung cấp, trả về danh sách foody_service_id của city_id đó
    if city_id is not None:
        return city_to_foody_dict.get(city_id, [])

    # Nếu không, trả về toàn bộ dictionary
    return city_to_foody_dict

# Ví dụ sử dụng hàm:
# Lấy toàn bộ dict
all_cities = get_foody_service_id()
print(all_cities)

# Lấy foody_service_id cho city_id cụ thể (ví dụ city_id = 1)
foody_ids = get_foody_service_id(city_id=257)
print(foody_ids)

{0: [], 257: [1, 5, 6, 11, 7, 4, 13, 17, 14, 16, 12], 259: [1, 4], 261: [1, 5, 4], 263: [1, 13, 5, 7, 16, 17, 14, 15, 4], 265: [1, 11, 7, 5, 6, 4, 12, 13, 17, 15, 14, 16], 267: [1, 4, 5], 269: [1, 4], 256: [1, 5, 4], 271: [1, 5, 6, 7, 17, 16, 40, 4], 272: [1, 5, 17, 4], 273: [1, 5, 6, 11, 7, 4, 13, 14, 17, 15, 12, 16], 274: [1, 5, 17], 275: [1, 4, 5], 277: [1, 4, 5], 217: [1, 2, 4, 3, 0, 5, 6, 7, 10, 11, 9, 8, 12, 13, 20, 15, 14, 17, 40], 218: [1, 0, 2, 5, 6, 4, 11, 7, 10, 12, 13, 20, 14, 17, 15], 219: [1, 5, 6, 7, 4, 10, 12, 13, 20, 14, 17, 16, 15], 220: [1, 5, 6, 11, 7, 4, 13, 20, 14, 17, 16, 12, 10], 221: [1, 5, 6, 11, 7, 4, 13, 20, 14, 17, 15, 12, 16], 222: [1, 5, 6, 11, 7, 4, 13, 17, 14, 15, 12, 16], 223: [1, 5, 6, 4, 7, 12, 13, 17, 14, 15, 16], 224: [1, 5, 17], 225: [1, 4, 5], 228: [1, 6, 7, 17, 16, 5, 40, 4], 229: [1, 4, 5], 230: [1, 5, 4, 6, 7, 12, 16, 17, 15, 14], 231: [1, 5, 17], 232: [1, 4], 233: [1, 5, 17, 4], 234: [1, 4, 5], 236: [1, 5, 17, 4], 238: [15, 18, 19, 1111, 1112

In [39]:
def get_foody_service_id(delivery_id):

    url = "https://gappapi.deliverynow.vn/api/delivery/get_browsing_infos"

    headers = {
    '152f5e39': '>9c0.F;&6Q=$\'+8X1;ce$pUP)',
    '32e77db7': 'CF_soOrp!LTWp_I%8?,o^,E80',
    'a56be009': 'gboekVPsd_\\=f.8kG/TqCnR\'b',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7,fr-FR;q=0.6,fr;q=0.5',
    'content-type': 'application/json;charset=UTF-8',
    'e759b938': 'TiPBD345FNVR<)g;f:l\'&:09bDposi4OX\\C="K(TW[DHh>64\'_J*49+PXmWXijO9*I*_ZVldm!=_dD7C$Tpb!-WG@!OM&E\\0r`aQ9\\/UM^*Yca&2emQE_1[)C2tUQFqok;ghCZRp]W@VnHXCNk5QZ:iX@4%R%ieWc<ARB1TUVYRKtLG6l7)IiZ:He3)"HS`/0PK6l7)IiZ:He3)"HS`/0PK',
    'origin': 'https://shopeefood.vn',
    'priority': 'u=1, i',
    'referer': 'https://shopeefood.vn/',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'x-foody-access-token': 'a694c06d56be4023c002f0215146beb18d113a24f6b5b8d4d218e19ba76cd9741b8f170f5ad1a719deb19b3b496dffcab3267a94eef120508d7046f4ece4ff7d',
    'x-foody-api-version': '1',
    'x-foody-app-type': '1004',
    'x-foody-client-id': '',
    'x-foody-client-language': 'vi',
    'x-foody-client-type': '1',
    'x-foody-client-version': '3.0.0',
    'x-sap-ri': '43bbcc6669f1a132f57ffa30db9efb3a9833bc0374be366c'
    }

    # Tạo payload chứa delivery_id
    payload = json.dumps({
        "delivery_ids": [delivery_id]
    })

    # Gửi yêu cầu POST
    response = requests.post(url, headers=headers, data=payload)

    # Xử lý phản hồi JSON
    data_json = response.json()

    # Tìm foody_service_id cho delivery_id
    delivery_infos = data_json.get('reply', {}).get('delivery_infos', [])
    for item in delivery_infos:
        if item.get('delivery_id') == delivery_id:
            return item.get('foody_service_id')

    # Trả về None nếu không tìm thấy foody_service_id cho delivery_id
    return None

# Ví dụ sử dụng hàm:
delivery_id = 112538
foody_service_id = get_foody_service_id(delivery_id)
print(f"Foody Service ID for delivery ID {delivery_id}: {foody_service_id}")

Foody Service ID for delivery ID 112538: 5


In [40]:
# Hàm để lấy món ăn
def get_dishes(request_id):
    url = f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?id_type=2&request_id={request_id}"

    headers = {
        '2a6adc4f': '6:<HW\\jkaG97HG(Ipl.hEIo=^',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'c60c0ff6': '35(3^E\\_.Q&UfMRZBW7Q(Z0o*',
        'f975249': '`=PkD#^YCfQ$V?Ij%DUh<6.\'ke#r"jk<_8SWm\'2(lp8Nr0$/URU2\'Dq)T]pK4n4.oUOJ.L"ULG/oltnTJj"Hl4#lKmP0lQFhY[e<YiS.?iR*m>HSTFD[3nDrc+EWp[YEdVa4u_I2V$;kA<MHGK")9Ic)HgAEX;T^88Zn:33"L>!G$:*nWQKNTU,%+>R+][!CUu&^PU/)q2H006L;\\#',
        'origin': 'https://shopeefood.vn',
        'priority': 'u=1, i',
        'referer': 'https://shopeefood.vn/',
        'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'x-foody-access-token': '',
        'x-foody-api-version': '1',
        'x-foody-app-type': '1004',
        'x-foody-client-id': '',
        'x-foody-client-language': 'vi',
        'x-foody-client-type': '1',
        'x-foody-client-version': '3.0.0',
        'x-sap-ri': '26b9c86624210819913d7a25f45e428e832591dc6f01ad2'
    }

    response = requests.get(url, headers=headers)
    data_json = response.json()
    dishes = data_json['reply']['menu_infos']
    all_dishes = []
    for menu_info in dishes:
        for dish in menu_info['dishes']:
            all_dishes.append(dish['name'])
    return all_dishes

def process_delivery_ids(delivery_ids, thread_id):
    print(f"Thread {thread_id} started.")
    results = []

    for delivery_id in delivery_ids:
        try:
            address, name = get_nameres(delivery_id)
            dishes = get_dishes(delivery_id)
            results.append({
                'delivery_id': delivery_id,
                'foody_service_ids':get_foody_service_id(delivery_id)
                ,
                'address': address,
                'name': name,
                'dishes': dishes,
                'city': city_name,
                'district': district_name
            })
            collection.insert_many(results)

        except Exception as e:
            results.append({
                'delivery_id': delivery_id,
                'error': str(e)
            })
            results.append(result)
    # Print results for this thread
    for result in results:
        if 'error' in result:
            print(f"Thread {thread_id} - Error with delivery_id {result['delivery_id']}: {result['error']}")
        else:
            print(f"Thread {thread_id} - Quán ăn: {result['name']}, Địa chỉ: {result['address']}")
            print(f"Menu của quán ăn {result['name']}:")
            for dish in result['dishes']:
                print(f"  - {dish}")

    print(f"Thread {thread_id} finished.")

# Function to split a list into smaller chunks
def split_list(lst, n):
    avg = len(lst) // n
    rem = len(lst) % n
    result = []
    start = 0
    for i in range(n):
        end = start + avg + (1 if i < rem else 0)
        result.append(lst[start:end])
        start = end
    return result

# URL của API meta để lấy city_id và district_id
url_meta = "https://gappapi.deliverynow.vn/api/meta/get_metadata"

# Headers và payload của yêu cầu meta
payload_meta = {}
headers_meta = {
    '1f64a3d4': 'J20t%[hF6amEo2!b_`J@m7p/0',
    '21f5f5c': '=/*H$Ds!c\\&TJZfcqeb>"O-V&',
    '9d0d791e': 'TGdh0dtl51:5I,VHG\\$NWRXG49,q\\hbpFY#c2lrK`h%^g]6CZ.M[E)k9Vg>^Z\'T32#"W&/rL5mIW)8=5EH,qBjp4H1p93^G%n0rr\'-6p,2[[!9faD"mR?Z9:=HI<>iAQ[C$3tL;WtO0g0\'g"oZq/ek77Ki@T]a$k\\PUT!gQup_imah^i:C_`$"On;c$!C-fbWL=T8`^BWKV+pYIG@',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'origin': 'https://shopeefood.vn',
    'priority': 'u=1, i',
    'referer': 'https://shopeefood.vn/',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'x-foody-access-token': '757c5659472f485202b4aad9f88042788da9938ebf0c69c11a1ad6576ad01fea43ed56559bd978a2cf73df3b5d2e865fcf523ef7973799167d30cf464c759cbc',
    'x-foody-api-version': '1',
    'x-foody-app-type': '1004',
    'x-foody-client-id': '',
    'x-foody-client-language': 'vi',
    'x-foody-client-type': '1',
    'x-foody-client-version': '3.0.0',
    'x-sap-ri': '9ffcc8660b91f7aea3b4813fc8f2add624b888bc93641541'
}

# Gửi yêu cầu GET tới API meta
response_meta = requests.get(url_meta, headers=headers_meta, data=payload_meta)

# Chuyển đổi phản hồi thành từ điển
try:
    response_data_meta = response_meta.json()
except json.JSONDecodeError:
    response_data_meta = {}

# Lấy thông tin các thành phố và quận
if isinstance(response_data_meta, dict):
    country = response_data_meta.get('reply', {}).get('country', {})

    # Kiểm tra nếu 'country' là từ điển
    if isinstance(country, dict):
        cities = country.get('cities', [])

        # Kiểm tra nếu 'cities' là danh sách
        if isinstance(cities, list):
            for city in cities:
                city_id = city.get('id', None)
                city_name = city.get('name', 'N/A')

                # Bỏ qua nếu city_id không hợp lệ
                if city_id is None:
                    continue

                print(f"Thành phố: ID = {city_id}, Tên = {city_name}")

                # Lấy thông tin các quận
                districts = city.get('districts', [])
                if isinstance(districts, list):
                    for district in districts:
                        district_id = district.get('district_id', None)
                        district_name = district.get('name', 'N/A')

                        # Bỏ qua nếu district_id không hợp lệ
                        if district_id is None:
                            continue

                        print(f"  Quận: ID = {district_id}, Tên = {district_name}")

                        # URL của API get_browsing_ids
                        url_browsing = "https://gappapi.deliverynow.vn/api/delivery/get_browsing_ids"

                        # Lặp qua 40 lần với sort_type từ 1 đến 40
                        delivery_ids = []
                        for i in range(40):
                            sort_type = i + 1

                            # Bỏ qua những sort_type không cần thiết (điều kiện tùy chọn)
                            if sort_type > 30:  # Ví dụ: chỉ xử lý các sort_type <= 30
                                continue
                            
                            #Lấy foody_services_ids theo delivery_id
                            foody_services_id = get_foody_service_id(delivery_id)

                            # Payload dưới dạng chuỗi JSON
                            payload_browsing = json.dumps({
                                "city_id": city_id,
                                "sort_type": sort_type,
                                "foody_services_id": foody_service_id,
                                "district_ids": [district_id]
                            })

                            # Headers của yêu cầu get_browsing_ids
                            headers = {
                                '3dee4201': '%RW7))I#Y:nCN\'job9OYY@]t#',
                                '90c493a': '%*=D8M",(Z(@c!=@YRGS[.]s0A\';_d\\WncZ;uWXiR9&^g!sBMW%^M5UT2Y_kVS2^C9b:opiui9LmBpc-Cd.]<V4,@>iG8aormnYm#3\'_.(Oft_)2S/\'RgicH/e7YY+)Yq-8ZGsX!L-nqCd;:0B=$aEf3j>.KpcMO+X&HWGX5>P\\W01LlU,/1Yh[PFMY(`.c^HEblU,/1Yh[PFMY(`.c^HEb',
                                'a29960': 'q6%`NP]OjB[Zo3Z2BmRU(015Y',
                                'accept': 'application/json, text/plain, */*',
                                'accept-language': 'en-US,en;q=0.9',
                                'content-type': 'application/json;charset=UTF-8',
                                'dc88c393': '@=uS5Tj#Z#Qn*4"a\\lSID"Z<^',
                                'origin': 'https://shopeefood.vn',
                                'priority': 'u=1, i',
                                'referer': 'https://shopeefood.vn/',
                                'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
                                'sec-ch-ua-mobile': '?0',
                                'sec-ch-ua-platform': '"Windows"',
                                'sec-fetch-dest': 'empty',
                                'sec-fetch-mode': 'cors',
                                'sec-fetch-site': 'cross-site',
                                'x-foody-access-token': 'f0b51c7e4414706d23242f3b32c8f7bbb499446801aee03fcbe107b3e965cb4e0c830ff32558ece154d6f86fc124962760a6d07ce9a7a3e080e61dd7a24bd47a',
                                'x-foody-api-version': '1',
                                'x-foody-app-type': '1004',
                                'x-foody-client-id': '',
                                'x-foody-client-language': 'vi',
                                'x-foody-client-type': '1',
                                'x-foody-client-version': '3.0.0',
                                'x-sap-ri': 'e1cdc9668d3213f14a1b943eb6ef35ec842f8788fe0c3137'
                            }

                            # Gửi yêu cầu POST tới API get_browsing_ids
                            response_browsing = requests.post(url_browsing, headers=headers, data=payload_browsing)

                            # Kiểm tra và lưu trữ phản hồi hợp lệ
                            try:
                                response_data_browsing = response_browsing.json()
                                
                                # Kiểm tra phản hồi có chứa 'reply' và 'delivery_ids'
                                if response_data_browsing.get('reply') and 'delivery_ids' in response_data_browsing['reply']:
                                    delivery_ids.extend(response_data_browsing['reply']['delivery_ids'])
                            except json.JSONDecodeError:
                                pass  # Bỏ qua các lỗi không hợp lệ

                            # Đợi 1 giây trước khi gửi yêu cầu tiếp theo

                        # Xử lý các delivery_ids
                        # if delivery_ids:
                            # process_delivery_ids(delivery_ids)
                            split_delivery_ids = split_list(delivery_ids, 16)
                            threads = []

                            for i, sublist in enumerate(split_delivery_ids):
                                thread = threading.Thread(target=process_delivery_ids, args=(sublist, i + 1))
                                thread.start()
                                threads.append(thread)

                            # Wait for all threads to complete
                            for thread in threads:
                                thread.join()

                            print("All threads have finished processing.")
        else:

            print("Dữ liệu về thành phố không phải là danh sách.")
    else:
        print("'country' không phải là từ điển.")
else:
    print("Dữ liệu phản hồi không phải là từ điển.")


Thành phố: ID = 217, Tên = TP. HCM
  Quận: ID = 1, Tên = Quận 1
Thread 1 started.
Thread 2 started.
Thread 3 started.
Thread 4 started.
Thread 5 started.
Thread 6 started.
Thread 7 started.
Thread 8 started.
Thread 9 started.
Thread 10 started.
Thread 11 started.
Thread 12 started.
Thread 13 started.
Thread 14 started.
Thread 15 started.
Thread 16 started.


Exception in thread Thread-5716 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\_csot.py", line 108, in csot_wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\collection.py", line 742, in insert_many
    blk.execute(write_concern, session, _Op.INSERT)
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\bulk.py", line 595, in execute
    return self.execute_command(generator, write_concern, session, operation)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\bulk.py", line 462, in execute_command
    _raise_bulk_write_error(full_result)
  File "d:\s

All threads have finished processing.
Thread 1 started.
Thread 2 started.
Thread 3 started.
Thread 4 started.
Thread 5 started.
Thread 6 started.
Thread 7 started.
Thread 8 started.
Thread 9 started.
Thread 10 started.
Thread 11 started.
Thread 12 started.
Thread 13 started.
Thread 14 started.
Thread 15 started.
Thread 16 started.


Exception in thread Thread-5731 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
Exception in thread Thread-5723 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\_csot.py", line 108, in csot_wrapper
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\_csot.py", line 108, in csot_wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\collection.py", line 742, in insert_many
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\collection.py", lin

All threads have finished processing.
Thread 1 started.
Thread 2 started.
Thread 3 started.
Thread 4 started.
Thread 5 started.
Thread 6 started.
Thread 7 started.
Thread 8 started.
Thread 9 started.
Thread 10 started.
Thread 11 started.
Thread 12 started.
Thread 13 started.
Thread 14 started.
Thread 15 started.
Thread 16 started.


...

All threads have finished processing.
Thread 1 started.
Thread 2 started.
Thread 3 started.
Thread 4 started.
Thread 5 started.
Thread 6 started.
Thread 7 started.
Thread 8 started.
Thread 9 started.
Thread 10 started.
Thread 11 started.
Thread 12 started.
Thread 13 started.
Thread 14 started.
Thread 15 started.
Thread 16 started.


Exception in thread Thread-5812 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 45, in process_delivery_ids
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\921444776.py", line 43, in get_nameres
KeyError: 'delivery_detail'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
Exception in thread Thread-5811 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 45, in process_delivery_ids
    _threading_Thread_run(self)
  File "C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in r

KeyboardInterrupt: 

Exception in thread Thread-5804 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
Exception in thread Thread-5814 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
    _raise_bulk_write_error(full_result)
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\bulk.py", line 165, in _raise_bulk_write_error
Exception in thread Thread-5807 (process_delivery_ids):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_3156\2681381640.py", line 56, in process_delivery_ids
    raise BulkWriteError(full_result)
pymongo.errors.BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: shopeefood.shopeefood index: _id_ dup ke

    return self.execute_command(generator, write_concern, session, operation)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\bulk.py", line 462, in execute_command
    _threading_Thread_run(self)
  File "C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    _raise_bulk_write_error(full_result)
  File "d:\study\BigData\CrawlShopee\web-crawler\env\Lib\site-packages\pymongo\bulk.py", line 165, in _raise_bulk_write_error
    raise BulkWriteError(full_result)
pymongo.errors.BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: shopeefood.shopeefood index: _id_ dup key: { _id: ObjectId('66ccc3cba93dd2c824b4fd30') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('66ccc3cba93dd2c824b4fd30')}, 'op': {'delivery_id': 7894, 'foody_service_ids': 